In [1]:
import pandas as pd
import model_rf_base
import os, sys
sys.path.append(os.path.abspath("../../etc/"))
import config

In [2]:
df = pd.read_csv("./data/standardized_set.csv")

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from enum import Enum
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA

class rf_antibody:
    def __init__(self, target: str, df: pd.DataFrame) -> None:
        self.exclude = [f'is_binding_{target}', f'is_neutral_{target}', 'name']
        self.rf_classifier_neutral: RandomForestClassifier
        self.rf_classifier_binding: RandomForestClassifier
        self.target = target
        self.df = df

    def train(self, X:pd.DataFrame, y: pd.Series):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        rf_antibody = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
        rf_antibody.fit(X_train, y_train)
        y_pred = rf_antibody.predict(X_test)
        return rf_antibody, classification_report(y_test, y_pred)
        
    def get_x_y(self, label: str):
        bool_not_unknown = self.df[label] != 2
        X = self.df[bool_not_unknown].drop(columns=self.exclude)
        y = self.df[bool_not_unknown][label]
        return X, y
        
    def train_neutral(self):
        label = f'is_neutral_{self.target}'
        X, y = self.get_x_y(label)
        self.rf_classifier_neutral, report = self.train(X, y)
        return self.rf_classifier_neutral, report
    
    def train_binding(self):
        label = f'is_binding_{self.target}'
        X, y = self.get_x_y(label)
        self.rf_classifier_binding, report = self.train(X, y)
        return self.rf_classifier_binding, report
        
        

In [6]:
model = rf_antibody(config.TARGET, df)

In [7]:
_, report = model.train_binding()

ValueError: could not convert string to float: '368.02a.C.0069'

In [ ]:
print(report)

              precision    recall  f1-score   support

           0       1.00      0.08      0.15       126
           1       0.95      1.00      0.97      2254

    accuracy                           0.95      2380
   macro avg       0.98      0.54      0.56      2380
weighted avg       0.95      0.95      0.93      2380



In [ ]:
_, report = model.train_neutral()

In [ ]:
print(report)

              precision    recall  f1-score   support

           0       0.76      0.45      0.57       612
           1       0.75      0.92      0.83      1113

    accuracy                           0.76      1725
   macro avg       0.76      0.69      0.70      1725
weighted avg       0.76      0.76      0.74      1725

